In [1]:
# Imports
import os
import pandas as pd
import numpy as np
from scipy import signal
from utilities import *

### Read whole data and store them in seperate files

#### This cell code is store the dataset after filter and fft process, and they are already exsit in the data files.

   So it is not necessary to run this cell unless the datasets have been deleted or the filter frequency want to be changed.

In [2]:
for j in range (1,33):
    file_prefix = './data/Trial'
    acc_df = pd.read_csv(file_prefix + str(j) + '/Myo_accelerometer.csv',index_col=None, header=None)

    acc_temp = acc_df.values
    acc_dataOut = np.zeros((len(acc_temp),3))
    acc_dataOut_fft = np.zeros((len(acc_temp),3))
    for p in range(3):
        acc_dataOut[:,p] = butter_lowpass_filter(acc_df.iloc[:,p+1], cutoff, fss, order) #cutoff
        acc_dataOut[:,p] = acc_dataOut[:,p]/max(abs(acc_dataOut[:,p]))
        acc_dataOut_fft[:,p] = np.fft.fft(acc_dataOut[:,p]).real

    acc_data_filter = np.concatenate([acc_temp[:,0:1], acc_dataOut], axis=1)
    acc_data_filter_fft = np.concatenate([acc_temp[:,0:1], acc_dataOut_fft], axis=1)

    acc_df = pd.DataFrame.from_records(acc_data_filter)
    acc_df_fft = pd.DataFrame.from_records(acc_data_filter_fft)



    gyro_df = pd.read_csv(file_prefix + str(j) + '/Myo_gyro.csv',index_col=None, header=None)

    gyro_temp = gyro_df.values
    gyro_dataOut = np.zeros((len(gyro_temp),3))
    gyro_dataOut_fft = np.zeros((len(gyro_temp),3))
    for p in range(3):
        gyro_dataOut[:,p] = butter_lowpass_filter(gyro_df.iloc[:,p+1], cutoff, fss, order)
        gyro_dataOut[:,p] = gyro_dataOut[:,p]/max(abs(gyro_dataOut[:,p]))
        gyro_dataOut_fft[:,p] = np.fft.fft(gyro_dataOut[:,p]).real

    gyro_data_filter = np.concatenate([gyro_temp[:,0:1], gyro_dataOut], axis=1)
    gyro_data_filter_fft = np.concatenate([gyro_temp[:,0:1], gyro_dataOut_fft], axis=1)

    gyro_df = pd.DataFrame.from_records(gyro_data_filter)
    gyro_df_fft = pd.DataFrame.from_records(gyro_data_filter_fft)


    euler_df = pd.read_csv(file_prefix + str(j) + '/Myo_orientationEuler.csv',index_col=None, header=None)

    euler_temp = euler_df.values
    euler_dataOut = np.zeros((len(euler_temp),3))
    euler_dataOut_fft = np.zeros((len(euler_temp),3))
    for p in range(3):
        euler_dataOut[:,p] = butter_lowpass_filter(euler_df.iloc[:,p+1], cutoff, fss, order)
        euler_dataOut[:,p] = euler_dataOut[:,p]/max(abs(euler_dataOut[:,p]))
        euler_dataOut_fft[:,p] = np.fft.fft(euler_dataOut[:,p]).real

    euler_data_filter = np.concatenate([euler_temp[:,0:1], euler_dataOut], axis=1)
    euler_data_filter_fft = np.concatenate([euler_temp[:,0:1], euler_dataOut_fft], axis=1)


    euler_df = pd.DataFrame.from_records(euler_data_filter)
    euler_df_fft = pd.DataFrame.from_records(euler_data_filter_fft)


    emg_df = pd.read_csv(file_prefix + str(j) + '/Myo_emg.csv',index_col=None, header=None)

    temp = emg_df.values
    dataOut = np.zeros((len(temp),8))
    dataOut_fft = np.zeros((len(temp),8))
    for p in range(8):
        dataOut[:,p] = emg_butter_lowpass_filter(emg_df.iloc[:,p+1], emg_cutoff, fs, order) # ,emg_cutoff
        dataOut[:,p] = dataOut[:,p]/max(abs(dataOut[:,p]))
        dataOut_fft[:,p] = np.fft.fft(dataOut[:,p]).real

    data_filter = np.concatenate([temp[:,0:1], dataOut], axis=1)
    data_filter_fft = np.concatenate([temp[:,0:1], dataOut_fft], axis=1)

    emg_df = pd.DataFrame.from_records(data_filter)
    emg_df_fft = pd.DataFrame.from_records(data_filter_fft)

    emg_df.to_csv(file_prefix + str(j) + '/emg_filter' + '.csv', index=False)
    emg_df_fft.to_csv(file_prefix + str(j) + '/emg_fft' + '.csv', index=False)

    gyro_df.to_csv(file_prefix + str(j) + '/gyro_filter' + '.csv', index=False)
    gyro_df_fft.to_csv(file_prefix + str(j) + '/gyro_fft' + '.csv', index=False)

    euler_df.to_csv(file_prefix + str(j) + '/orientationEuler_filter' + '.csv', index=False)
    euler_df_fft.to_csv(file_prefix + str(j) + '/orientationEuler_fft' + '.csv', index=False)

    acc_df.to_csv(file_prefix + str(j) + '/accelerometer_filter' + '.csv', index=False)
    acc_df_fft.to_csv(file_prefix + str(j) + '/accelerometer_fft' + '.csv', index=False)

#### Only Filtered Dataset (This cell should be re-run every time when the labels are changed)

resample and segement window

In [3]:
"""" read and label the datasets of only Filter data """

name = ['gyro_filter','orientationEuler_filter','accelerometer_filter']
gyro = store_label_data(name[0])
Myo_orientationEuler = store_label_data(name[1])
Myo_accelerometer = store_label_data(name[2])
imu_data = pd.concat([gyro.iloc[:,0:4],Myo_orientationEuler.iloc[:,1:4],Myo_accelerometer.iloc[:,1:5]],axis=1)
imu_data = imu_data.iloc[:,1:]
name = ['emg_filter']
emg = store_label_data(name[0])
emg_data = emg.iloc[:,1:]

"""" resample the EMG data into the same length of IMU data saved as resample_data """
f0 = signal.resample(emg_data.iloc[:,0], len(imu_data)) # resample data to imu numbers
f1 = signal.resample(emg_data.iloc[:,1], len(imu_data))
f2 = signal.resample(emg_data.iloc[:,2], len(imu_data))
f3 = signal.resample(emg_data.iloc[:,3], len(imu_data))
f4 = signal.resample(emg_data.iloc[:,4], len(imu_data))
f5 = signal.resample(emg_data.iloc[:,5], len(imu_data))
f6 = signal.resample(emg_data.iloc[:,6], len(imu_data))
f7 = signal.resample(emg_data.iloc[:,7], len(imu_data))
resample_data = np.concatenate([f0[:,np.newaxis], f1[:,np.newaxis], f2[:,np.newaxis],f3[:,np.newaxis],
                                f4[:,np.newaxis], f5[:,np.newaxis], f6[:,np.newaxis], f7[:,np.newaxis]],axis=1)
resample_data = pd.DataFrame(resample_data)

In [4]:
"""" set the window size and segement IMU and EMG respectively  """
window_size = 10

# get the label and the label after encoding in one hot 
labels = extract_labels(imu_data,window_size)
onehot_labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)

##############      IMU         #################
data_len = imu_data.iloc[:,0].shape[0]
reshaped_x1 = imu_data.iloc[:(data_len//window_size)*window_size,0].values.reshape([data_len//window_size,window_size,1])
reshaped_x2 = imu_data.iloc[:(data_len//window_size)*window_size,1].values.reshape([data_len//window_size,window_size,1])
reshaped_x3 = imu_data.iloc[:(data_len//window_size)*window_size,2].values.reshape([data_len//window_size,window_size,1])
reshaped_x4 = imu_data.iloc[:(data_len//window_size)*window_size,3].values.reshape([data_len//window_size,window_size,1])
reshaped_x5 = imu_data.iloc[:(data_len//window_size)*window_size,4].values.reshape([data_len//window_size,window_size,1])
reshaped_x6 = imu_data.iloc[:(data_len//window_size)*window_size,5].values.reshape([data_len//window_size,window_size,1])
reshaped_x7 = imu_data.iloc[:(data_len//window_size)*window_size,6].values.reshape([data_len//window_size,window_size,1])
reshaped_x8 = imu_data.iloc[:(data_len//window_size)*window_size,7].values.reshape([data_len//window_size,window_size,1])
reshaped_x9 = imu_data.iloc[:(data_len//window_size)*window_size,8].values.reshape([data_len//window_size,window_size,1])
reshaped_segments_imu = np.concatenate([reshaped_x1, reshaped_x2, reshaped_x3,reshaped_x4, reshaped_x5, reshaped_x6,
                                   reshaped_x7, reshaped_x8, reshaped_x9],axis=2)

##############      EMG        #################
data_len = resample_data.iloc[:,0].shape[0]
reshaped_x1 = resample_data.iloc[:(data_len//window_size)*window_size,0].values.reshape([data_len//window_size,window_size,1])
reshaped_x2 = resample_data.iloc[:(data_len//window_size)*window_size,1].values.reshape([data_len//window_size,window_size,1])
reshaped_x3 = resample_data.iloc[:(data_len//window_size)*window_size,2].values.reshape([data_len//window_size,window_size,1])
reshaped_x4 = resample_data.iloc[:(data_len//window_size)*window_size,3].values.reshape([data_len//window_size,window_size,1])
reshaped_x5 = resample_data.iloc[:(data_len//window_size)*window_size,4].values.reshape([data_len//window_size,window_size,1])
reshaped_x6 = resample_data.iloc[:(data_len//window_size)*window_size,5].values.reshape([data_len//window_size,window_size,1])
reshaped_x7 = resample_data.iloc[:(data_len//window_size)*window_size,6].values.reshape([data_len//window_size,window_size,1])
reshaped_x8 = resample_data.iloc[:(data_len//window_size)*window_size,7].values.reshape([data_len//window_size,window_size,1])
reshaped_segments_emg = np.concatenate([reshaped_x1, reshaped_x2, reshaped_x3,reshaped_x4, reshaped_x5, reshaped_x6,
                                   reshaped_x7, reshaped_x8],axis=2)

In [5]:
# check the shape of each data and labels
reshaped_segments_imu.shape,reshaped_segments_emg.shape,labels.shape,onehot_labels.shape

((31524, 10, 9), (31524, 10, 8), (31524,), (31524, 4))

#### Filtered and FFT Dataset (This cell should be re-run every time when the labels are changed)

In [6]:
"""" read and label the datasets of Filtered and FFT data """

name = ['gyro_fft','orientationEuler_fft','accelerometer_fft']
gyro_fft = store_label_data(name[0])
Myo_orientationEuler_fft = store_label_data(name[1])
Myo_accelerometer_fft = store_label_data(name[2])
imu_data_fft = pd.concat([gyro_fft.iloc[:,0:4],Myo_orientationEuler_fft.iloc[:,1:4],Myo_accelerometer_fft.iloc[:,1:5]],axis=1)
imu_data_fft = imu_data_fft.iloc[:,1:]
name = ['emg_fft']
emg_fft = store_label_data(name[0])
emg_data_fft = emg_fft.iloc[:,1:]

"""" resample the EMG data into the same length of IMU data saved as resample_data """
f0 = signal.resample(emg_data_fft.iloc[:,0], len(imu_data_fft)) # resample data to imu numbers
f1 = signal.resample(emg_data_fft.iloc[:,1], len(imu_data_fft))
f2 = signal.resample(emg_data_fft.iloc[:,2], len(imu_data_fft))
f3 = signal.resample(emg_data_fft.iloc[:,3], len(imu_data_fft))
f4 = signal.resample(emg_data_fft.iloc[:,4], len(imu_data_fft))
f5 = signal.resample(emg_data_fft.iloc[:,5], len(imu_data_fft))
f6 = signal.resample(emg_data_fft.iloc[:,6], len(imu_data_fft))
f7 = signal.resample(emg_data_fft.iloc[:,7], len(imu_data_fft))
resample_data_fft = np.concatenate([f0[:,np.newaxis], f1[:,np.newaxis], f2[:,np.newaxis],f3[:,np.newaxis],
                                f4[:,np.newaxis], f5[:,np.newaxis], f6[:,np.newaxis], f7[:,np.newaxis]],axis=1)
resample_data_fft = pd.DataFrame(resample_data_fft)

In [7]:
window_size = 10

##############      IMU         #################
data_len = imu_data_fft.iloc[:,0].shape[0]
reshaped_x1 = imu_data_fft.iloc[:(data_len//window_size)*window_size,0].values.reshape([data_len//window_size,window_size,1])
reshaped_x2 = imu_data_fft.iloc[:(data_len//window_size)*window_size,1].values.reshape([data_len//window_size,window_size,1])
reshaped_x3 = imu_data_fft.iloc[:(data_len//window_size)*window_size,2].values.reshape([data_len//window_size,window_size,1])
reshaped_x4 = imu_data_fft.iloc[:(data_len//window_size)*window_size,3].values.reshape([data_len//window_size,window_size,1])
reshaped_x5 = imu_data_fft.iloc[:(data_len//window_size)*window_size,4].values.reshape([data_len//window_size,window_size,1])
reshaped_x6 = imu_data_fft.iloc[:(data_len//window_size)*window_size,5].values.reshape([data_len//window_size,window_size,1])
reshaped_x7 = imu_data_fft.iloc[:(data_len//window_size)*window_size,6].values.reshape([data_len//window_size,window_size,1])
reshaped_x8 = imu_data_fft.iloc[:(data_len//window_size)*window_size,7].values.reshape([data_len//window_size,window_size,1])
reshaped_x9 = imu_data_fft.iloc[:(data_len//window_size)*window_size,8].values.reshape([data_len//window_size,window_size,1])
reshaped_segments_imu_fft = np.concatenate([reshaped_x1, reshaped_x2, reshaped_x3,reshaped_x4, reshaped_x5, reshaped_x6,
                                   reshaped_x7, reshaped_x8, reshaped_x9],axis=2)

##############      EMG        #################
data_len = resample_data_fft.iloc[:,0].shape[0]
reshaped_x1 = resample_data_fft.iloc[:(data_len//window_size)*window_size,0].values.reshape([data_len//window_size,window_size,1])
reshaped_x2 = resample_data_fft.iloc[:(data_len//window_size)*window_size,1].values.reshape([data_len//window_size,window_size,1])
reshaped_x3 = resample_data_fft.iloc[:(data_len//window_size)*window_size,2].values.reshape([data_len//window_size,window_size,1])
reshaped_x4 = resample_data_fft.iloc[:(data_len//window_size)*window_size,3].values.reshape([data_len//window_size,window_size,1])
reshaped_x5 = resample_data_fft.iloc[:(data_len//window_size)*window_size,4].values.reshape([data_len//window_size,window_size,1])
reshaped_x6 = resample_data_fft.iloc[:(data_len//window_size)*window_size,5].values.reshape([data_len//window_size,window_size,1])
reshaped_x7 = resample_data_fft.iloc[:(data_len//window_size)*window_size,6].values.reshape([data_len//window_size,window_size,1])
reshaped_x8 = resample_data_fft.iloc[:(data_len//window_size)*window_size,7].values.reshape([data_len//window_size,window_size,1])
reshaped_segments_emg_fft = np.concatenate([reshaped_x1, reshaped_x2, reshaped_x3,reshaped_x4, reshaped_x5, reshaped_x6,
                                   reshaped_x7, reshaped_x8],axis=2)

In [8]:
reshaped_segments_imu_fft.shape,reshaped_segments_emg_fft.shape

((31524, 10, 9), (31524, 10, 8))